<small><i>Updated February 2023 - This notebook was created by [Santi Seguí](https://ssegui.github.io/). </i></small>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\mecheste\AppData\Local\Temp\ipykernel_17832\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Item association recommendation
### Example:
<img src="images/np3.png" width=70%>


In [2]:
#Let's read a dataset which contains several market baskets lists

# read data/grocieries.csv
def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

market_data = []
cont = 0
items = []
with open("./data/groceries.csv") as f:
    for l in f:
        market_data.append(l.rstrip().split(','))
        items = union(items,l.rstrip().split(','))

print("Number of different items", len(items))
print("Number of rows ", len(market_data))


print("An example:", market_data[3])

Number of different items 171
Number of rows  9835
An example: ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads']


One of the most simple ways to found association between product could be obtained as follows: $$score(Y|X) = \frac{X \ and \ Y}{X}$$

In [3]:
# Which is the top associated product with "yogurt"?

In [4]:
from collections import defaultdict

# Which is the top associated product with "yogurt"?
def top_associated_products(dataset, product,N = 5):
    d = defaultdict(lambda: 0) # dictionary for the items 
    product_times = 0
    for basket in dataset: # for each basket case 
        if product in basket:  # for those who contains the product
            product_times += 1
            for i in basket:   # x and y together
                if i != product: 
                    d[i] += 1  
    for k in d:
        d[k] =   float(d[k] / product_times) # (X and Y) / X
    sorted_list=sorted(d.items(), key=lambda x: x[1],reverse=True)
    return sorted_list[:N]

In [5]:
s = top_associated_products(market_data, 'yogurt',N = 3)
print(s)

[('whole milk', 0.40160349854227406), ('other vegetables', 0.3112244897959184), ('rolls/buns', 0.24635568513119532)]


In [6]:
s = top_associated_products(market_data, 'rice',N = 3)
print(s)

[('whole milk', 0.6133333333333333), ('other vegetables', 0.52), ('root vegetables', 0.41333333333333333)]


In [7]:
s = top_associated_products(market_data, 'rum',N = 3)
print(s)

[('whole milk', 0.38636363636363635), ('other vegetables', 0.3409090909090909), ('tropical fruit', 0.20454545454545456)]


### What happens? 
Is it a good measure? It has a problem with popular items...
<br>
Let's check this other formula:
$$score(Y|X) = \frac{ \frac{X \ and \ Y}{X}} {  \frac{!X \ and \ Y}{!X} }  $$

In [8]:
def top_associated_products2(dataset, product,N = 5):
    prob_yx = defaultdict(lambda:0)
    not_x_y = defaultdict(lambda:0)
    x_y = defaultdict(lambda:0)
    x, not_x = 0,0

    for basket in dataset:
        if product in basket: 
            x += 1
            for i in basket:
                if i != product:
                    x_y[i] += 1.0
        else:
            not_x += 1 
            for i in basket:
                not_x_y[i] += 1

    for k in x_y:
        if (not_x_y[k] == 0): # special case
            prob_yx[k] = x
        else:
            prob_yx[k] = (x_y[k] * not_x)/(x + not_x_y[k])

    return sorted(prob_yx.items(), key = lambda x: x[1], reverse = True)[:N]

In [9]:
s = top_associated_products2(market_data, 'yogurt',N = 3)
print(s)

s = top_associated_products2(market_data, 'rice',N = 3)
print(s)

# Which is the top associated prouct with "rum"?
s = top_associated_products2(market_data, 'rum',N = 3)
print(s)

[('whole milk', 1398.6541691661669), ('baby food', 1372), ('other vegetables', 1268.8556882022472)]
[('canned fruit', 379.02912621359224), ('turkey', 325.3333333333333), ('hard cheese', 318.9542483660131)]
[('artif. sweetener', 264.6216216216216), ('frozen dessert', 199.81632653061226), ('liquor', 195.82)]


#### Let's check this last formula:
$$ score(Y|X) = \frac{P(X \ and \ Y)}{P(X)P(Y) }   $$

In [10]:
def top_associated_products3(dataset, product,N = 5):
    d = defaultdict(lambda:0)
    times_item = defaultdict(lambda:0)

    for basket in dataset:
        for item in basket:
            times_item[item] += 1
            if product in basket:
                for y in basket:
                    if y != product:
                        d[y] += 1

    for k in d:
        d[k] = (d[k]/len(dataset)) / ((times_item[k]/len(dataset)) * (times_item[product]/len(dataset)))
        
    sorted_list = sorted(d.items(), key = lambda x : x[1], reverse = True)

    return sorted(d.items(), key = lambda x: x[1], reverse = True)[:N]

In [11]:
print(top_associated_products3(market_data,'yogurt',N = 3))
print(top_associated_products3(market_data,'rice',N = 3))
print(top_associated_products3(market_data,'rum',N = 3))

[('baby food', 186.37755102040816), ('kitchen utensil', 44.80229591836734), ('rice', 34.12142857142857)]
[('decalcifier', 236.04000000000002), ('salad dressing', 229.48333333333335), ('specialty vegetables', 223.69803921568626)]
[('cooking chocolate', 259.28636363636366), ('specialty vegetables', 144.63235294117646), ('artif. sweetener', 132.71661931818184)]


### Exercice: Let's apply Association Rules on MovieLens Dataset

In [12]:
#NETFLIX REAL 50.000.000 usuaris and 100.000 items
%autosave 150
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt

# Load Data set
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/ml-1m/users.dat', sep='::', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('./data/ml-1m/ratings.dat', sep='::', names=r_cols)

# the movies file contains columns indicating the movie's genres
# let's only load the first three columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date',]
movies = pd.read_csv('./data/ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3), encoding='latin-1')

# Construcció del DataFrame
data = pd.merge(pd.merge(ratings, users), movies)
data = data[['user_id','title', 'movie_id','rating','release_date','sex','age']]

n_users = data.user_id.nunique()
n_items = data.movie_id.nunique()
print("La BD has "+ str(data.shape[0]) +" ratings")
print("La BD has ", n_users," users")
print("La BD has ", n_items, " movies")
data.head()


Autosaving every 150 seconds


C:\Users\mecheste\AppData\Local\Temp\ipykernel_17832\2948232862.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv('./data/ml-1m/users.dat', sep='::', names=u_cols)
C:\Users\mecheste\AppData\Local\Temp\ipykernel_17832\2948232862.py:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('./data/ml-1m/ratings.dat', sep='::', names=r_cols)
C:\Users\mecheste\AppData\Local\Temp\ipykernel_17832\2948232862.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are int

La BD has 1000209 ratings
La BD has  6040  users
La BD has  3706  movies


,user_id,title,movie_id,rating,release_date,sex,age
0,1,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama,1,F
1,2,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama,56,M
2,12,One Flew Over the Cuckoo's Nest (1975),1193,4,Drama,25,M
3,15,One Flew Over the Cuckoo's Nest (1975),1193,4,Drama,25,M
4,17,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama,50,M


In [13]:
# Step 1: convert dataset

# Step 2: Apply previous methods

In [14]:
movies_list = []
users = set(data.user_id)
for user in users:
    df = data[(data.user_id == user)&(data.rating>=4)]
    movies_list.append(list(df.title.values))

In [15]:
s = top_associated_products(movies_list, 'Titanic (1997)',N = 5)
print(s)
print()
s = top_associated_products(movies_list, 'Star Wars: Episode IV - A New Hope (1977)',N = 5)
print(s)

[('Shawshank Redemption, The (1994)', 0.5680539932508436), ('Silence of the Lambs, The (1991)', 0.562429696287964), ('Saving Private Ryan (1998)', 0.5601799775028121), ('Sixth Sense, The (1999)', 0.5568053993250843), ('Forrest Gump (1994)', 0.5500562429696289)]

[('Star Wars: Episode V - The Empire Strikes Back (1980)', 0.7475209763539283), ('Raiders of the Lost Ark (1981)', 0.6357742181540809), ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.6052631578947368), ('Matrix, The (1999)', 0.5644546147978642), ('Terminator 2: Judgment Day (1991)', 0.5110602593440122)]


In [16]:
data.groupby(by = 'title').count().sort_values(by = 'user_id', ascending = False)

,user_id,movie_id,rating,release_date,sex,age
title,,,,,,
American Beauty (1999),3428,3428,3428,3428,3428,3428
Star Wars: Episode IV - A New Hope (1977),2991,2991,2991,2991,2991,2991
Star Wars: Episode V - The Empire Strikes Back (1980),2990,2990,2990,2990,2990,2990
Star Wars: Episode VI - Return of the Jedi (1983),2883,2883,2883,2883,2883,2883
Jurassic Park (1993),2672,2672,2672,2672,2672,2672
...,...,...,...,...,...,...
Target (1995),1,1,1,1,1,1
I Don't Want to Talk About It (De eso no se habla) (1993),1,1,1,1,1,1
An Unforgettable Summer (1994),1,1,1,1,1,1


## APRIORI Algorithm
Typically, association rules are considered interesting if they satisfy both a minimum support threshold and a minimum confidence threshold

![alt apriori](images/apriori.png)

<b>Apriori principle</b>: Any subset of a frequent itemset must be frequent

> Step 1: Find the frequent itemsset: the set of items that have minimum support.
> -  A subset of a frequent itemset must also be a frequent itemset  i.e. if {1,2} is a frequent itemset, both {1} and {2} should be a frequent itemset
> - Iteratively find frequent itemsets with cardinality from 1 to k (k-itemset)

> Step 2: Use the frequent itemsets to generate association rules

![alt apriori2](images/apriori2.png)

Reference : 
[Fast algorithms for mining association rules](http://www-cgi.cs.cmu.edu/afs/cs.cmu.edu/Web/People/ngm/15-721/summaries/12.pdf)


In [17]:
from itertools import chain, combinations

def dataFromFile(fname):
    """Function which reads from the file and yields a generator"""
    file_iter = open(fname, 'r')
    for line in file_iter:
        line = line.strip().rstrip(',')                         # Remove trailing comma
        record = frozenset(line.split(','))
        yield record
                
def getItemSetTransactionList(data_iterator):
    """Generate 1-itemSets"""
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList
                
def apriori(data_iterator, min_support, min_confidence, method = 'confidence'):
    """A-priori method"""
    def returnItemsWithMinSupport(itemSet, transactionList, min_support, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= min_support:
                        _itemSet.add(item)

        return _itemSet
    
    def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])
    
    def getSupport(item):
        """local function which Returns the support of an item"""
        return float(freqSet[item])/len(transactionList)
    
    def subsets(arr):
        """ Returns non empty subsets of arr"""
        return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

    
    itemSet, transactionList = getItemSetTransactionList(data_iterator)
    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy min_support
    
    assocRules = dict()
    
    # Step 1: Find the frequent itemsset: the set of items that have minimum support.
    oneCSet = returnItemsWithMinSupport(itemSet,transactionList,min_support,freqSet)
    currentLSet = oneCSet
    
    k = 2
    while(currentLSet != set([])): # while there is pontential new associations
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,transactionList,min_support, freqSet)
        currentLSet = currentCSet
        k = k + 1
    
    toRetItems = []
    for key, value in list(largeSet.items()):
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])


    ## Step 2: Use the frequent itemsets to generate association rules
    toRetRules = defaultdict(lambda: [])
    for key in list(largeSet.keys()):
        if key!=1: #for itemsets with two or more elements
            for item in largeSet[key]:
                for element in item:
                    remain = item-frozenset([element])
                    if method == 'confidence':
                        confidence = getSupport(item)/getSupport(remain)
                    elif method == 'lift':
                        confidence = getSupport(item)/(getSupport(remain)*getSupport(frozenset([element]))) ## lift
                        #print("NOT IMPLEMENTED")
                        #return [],[] 
                    else:
                        print("Not Valid Method")
                        return [],[] 
                    
                    if confidence >= min_confidence:
                        toRetRules[tuple(remain)].append((tuple([element]),confidence))
    
    return toRetItems, toRetRules

def printResults(items, rules, only_rules = True):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    if(len(items)>0):
        if(only_rules ==False):
            for item, support in sorted(items, key = lambda x: float(x[1])):
                print("item: %s , %.3f" % (str(item), support))
        print("\n------------------------ RULES:")
        for pre, post in sorted([(key, v) for key,values in rules.items() for v in values ],
                                       key=lambda x: float(x[1][1])):
            print("Rule: %s ==> %s , %.3f" % (str(pre), str(post[0]), post[1]))
        
                


In [18]:
inFile = dataFromFile('./data/groceries.csv')
min_support = 0.01
min_confidence = 0.3
items, rules =  apriori(inFile, min_support, min_confidence)
printResults(items, rules)


------------------------ RULES:
Rule: ('rolls/buns',) ==> ('whole milk',) , 0.308
Rule: ('berries',) ==> ('other vegetables',) , 0.309
Rule: ('whole milk', 'other vegetables') ==> ('root vegetables',) , 0.310
Rule: ('bottled water',) ==> ('whole milk',) , 0.311
Rule: ('yogurt',) ==> ('other vegetables',) , 0.311
Rule: ('dessert',) ==> ('other vegetables',) , 0.312
Rule: ('whole milk', 'bottled water') ==> ('other vegetables',) , 0.314
Rule: ('whole milk', 'rolls/buns') ==> ('other vegetables',) , 0.316
Rule: ('berries',) ==> ('yogurt',) , 0.318
Rule: ('pastry', 'whole milk') ==> ('other vegetables',) , 0.318
Rule: ('sausage',) ==> ('whole milk',) , 0.318
Rule: ('sugar',) ==> ('other vegetables',) , 0.318
Rule: ('coffee',) ==> ('whole milk',) , 0.322
Rule: ('curd',) ==> ('other vegetables',) , 0.323
Rule: ('curd',) ==> ('yogurt',) , 0.324
Rule: ('cream cheese ',) ==> ('yogurt',) , 0.325
Rule: ('sausage',) ==> ('rolls/buns',) , 0.326
Rule: ('frankfurter',) ==> ('rolls/buns',) , 0.326
Ru

In [19]:
print('yogurt ->', rules[tuple(frozenset(['yogurt']))])
print('chicken ->', rules[tuple(frozenset(['chicken']))])
print('napkins ->', rules[tuple(frozenset(['napkins']))])

yogurt -> [(('other vegetables',), 0.3112244897959184), (('whole milk',), 0.40160349854227406)]
chicken -> [(('whole milk',), 0.4099526066350711), (('other vegetables',), 0.4170616113744075)]
napkins -> [(('whole milk',), 0.37669902912621356)]


In [20]:
# Let's check it with LIFT 
inFile = dataFromFile('./data/groceries.csv')
min_support = 0.01
min_confidence = 1.8
items, rules =  apriori(inFile, min_support, min_confidence, method = 'lift')
#printResults(items, rules)

In [21]:
print('yogurt ->', rules[tuple(frozenset(['yogurt']))])
print('chicken ->', rules[tuple(frozenset(['chicken']))])
print('napkins ->', rules[tuple(frozenset(['napkins']))])

yogurt -> [(('berries',), 2.279847718904075), (('butter',), 1.894027335704924), (('curd',), 2.325615360648076), (('whipped/sour cream',), 2.0742509769865394), (('citrus fruit',), 1.8757521436092863), (('frozen vegetables',), 1.8489235017474217), (('fruit/vegetable juice',), 1.8551049111627773), (('cream cheese ',), 2.3306986729117876), (('tropical fruit',), 2.0004746084480303)]
chicken -> [(('root vegetables',), 2.32622064440829), (('other vegetables',), 2.1554392789633727)]
napkins -> [(('tropical fruit',), 1.831988033416121)]


<div class  = "alert alert-success"><b>Exercice: Create and Product Association Recommender with MovieLens Dataset</b><p>
Explain the obtained results and conclusions.
</div>



In [22]:
import csv

with open('top_movies.csv', 'w', encoding = 'UTF8') as f:
    writer = csv.writer(f)
    users = set(data.user_id)
    for user in users:
        df = df[data.user_id == user]
        if (df.shape[0] > 0):
            writer.writerow([str(x) for x in df.movie_id])

C:\Users\mecheste\AppData\Local\Temp\ipykernel_17832\711774915.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[data.user_id == user]


In [23]:
inFile = dataFromFile('./top_movies.csv')
min_support = 0.1
min_confidence = 0.3
items, rules =  apriori(inFile, min_support, min_confidence)
printResults(items, rules)